## 환경 세팅 




In [ ]:
%xmode Plain

# 1) torchmetrics 최신 + detection 의존성까지 한 번에
!pip install -U "torchmetrics[detection]"

# 2) Windows에서 pycocotools 빌드 문제 피하기용
!pip install pycocotools-windows

# (옵션) 대안 백엔드
# !pip install faster-coco-eval

!pip install -U ultralytics



Exception reporting mode: Plain


ERROR: Could not find a version that satisfies the requirement pycocotools-windows (from versions: none)
ERROR: No matching distribution found for pycocotools-windows


---

## (1-A) 일반 학습용 데이터 분할

### 목적

* 이미지-라벨 페어링을 점검하고, 무작위로 **train/val 세트를 분리**
* **모든 클래스가 검증 세트에 최소 1개 이상 포함되도록 보정**
* YOLO 학습용 **`data.yaml` 파일을 자동 생성**


### 주요 특징

1. 전체 이미지 및 라벨 파일 경로를 탐색하여 **1:1 페어링**
2. 라벨 없는 이미지, 이미지 없는 라벨을 탐지 후 **경고 출력**
3. 라벨 파일로부터 최대 클래스 id를 추출해 **자동으로 `nc` 계산**
4. `VAL_RATIO = 0.1` 비율로 학습/검증 데이터 분할
5. `ENSURE_EVERY_CLASS_IN_VAL = True`
   → 모든 클래스가 검증 세트에 **최소 1개 이상 포함되도록 보정**
6. 라벨 포맷 `[class x_center y_center w h]` 및 `[0~1]` 범위 **검증 로직 포함**
7. `train.txt`, `val.txt`, `data.yaml` 자동 저장


### 사용 시나리오

* 일반적인 **모델 학습 및 검증 구조 구성용**
* 과적합 방지 및 **일반화 성능 평가 목적**


### 출력 파일

| 파일명                 | 설명                               |
| ------------------- | -------------------------------- |
| `_splits/train.txt` | 학습용 이미지 목록                       |
| `_splits/val.txt`   | 검증용 이미지 목록                       |
| `data.yaml`         | YOLO 학습 구성 파일 (`nc`, `names` 포함) |


In [ ]:
import os, glob, random, yaml
from pathlib import Path

# ================================
# 설정
# ================================
IMG_DIR = r"E:\pytorch_env\YOLOv8x_dataset\images"
LBL_DIR = r"E:\pytorch_env\YOLOv8x_dataset\labels"
VAL_RATIO = 0.1
SEED = 777
random.seed(SEED)

# 유효 이미지 확장자
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# 경로/출력
DATA_ROOT = str(Path(IMG_DIR).parent)  # E:\pytorch_env\YOLOv8x_dataset
LIST_DIR = os.path.join(DATA_ROOT, "_splits")
os.makedirs(LIST_DIR, exist_ok=True)
train_list = os.path.join(LIST_DIR, "train.txt")
val_list   = os.path.join(LIST_DIR, "val.txt")
DATA_YAML  = os.path.join(DATA_ROOT, "data.yaml")

def stem(p): return Path(p).stem

# ================================
# 1) 이미지-라벨 페어링
# ================================
images = [p for ext in IMG_EXTS for p in glob.glob(os.path.join(IMG_DIR, f"**/*{ext}"), recursive=True)]
images = sorted(set(images))
labels = glob.glob(os.path.join(LBL_DIR, "**/*.txt"), recursive=True)
label_map = {stem(p): p for p in labels}

pairs = []
miss_lbl, miss_img = [], []

for img in images:
    s = stem(img)
    if s in label_map:
        pairs.append((img, label_map[s]))
    else:
        miss_lbl.append(img)

# 라벨은 있는데 이미지가 없는 경우 탐지
for s, p in label_map.items():
    # 이미지 경로 후보들(같은 stem, 다양한 확장자)
    found = any(os.path.exists(os.path.join(Path(IMG_DIR), *(Path(p).parts[len(Path(LBL_DIR).parts):-1]), s + e)) for e in IMG_EXTS)
    if not found:
        # 느슨하게 전체 이미지 폴더에서 stem 매칭
        if not any(stem(i) == s for i in images):
            miss_img.append(p)

print(f"총 이미지: {len(images)} | 총 라벨: {len(labels)} | 페어링 성공: {len(pairs)}")
if miss_lbl:
    print(f"[경고] 라벨 없는 이미지 {len(miss_lbl)}개 (학습에서 제외): 예) {miss_lbl[:3]}")
if miss_img:
    print(f"[경고] 이미지 없는 라벨 {len(miss_img)}개 (학습에서 제외): 예) {miss_img[:3]}")

assert len(pairs) > 0, "이미지-라벨 페어가 없습니다. 폴더 구성을 다시 확인하세요."

# ================================
# 2) 클래스 수 자동 추출 (라벨 최대 class id + 1)
# ================================
def max_class_id_from_label(txt_path):
    max_id = -1
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            try:
                cid = int(float(parts[0]))
                if cid > max_id:
                    max_id = cid
            except:
                pass
    return max_id

global_max = -1
for _, lbl in pairs:
    m = max_class_id_from_label(lbl)
    if m > global_max:
        global_max = m

assert global_max >= 0, "라벨 파일에서 클래스 id를 찾지 못했습니다."
nc = global_max + 1
print(f"추정 클래스 수 nc={nc}")

# ================================
# 3) 1차 분할 (무작위)
# ================================
random.shuffle(pairs)
val_count  = max(1, int(len(pairs) * VAL_RATIO))
val_pairs  = pairs[:val_count]
train_pairs = pairs[val_count:]

print(f"[초기] train: {len(train_pairs)} | val: {len(val_pairs)}")

# ================================
# 4) (옵션) val에 '클래스별 최소 1개' 보장
# ================================
from collections import defaultdict

def classes_in_label(txt_path):
    cs = set()
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            try:
                cs.add(int(float(parts[0])))
            except:
                pass
    return cs

# 희귀 클래스 보장 모드 ON/OFF (원하면 False)
ENSURE_EVERY_CLASS_IN_VAL = True

if ENSURE_EVERY_CLASS_IN_VAL:
    pair_classes = [(img, lbl, classes_in_label(lbl)) for img, lbl in pairs]
    random.shuffle(pair_classes)
    target_val = max(1, int(len(pair_classes) * VAL_RATIO))

    val_pairs, train_pairs = [], []
    class_seen = defaultdict(bool)

    # (1) 각 클래스별 첫 샘플을 val에 우선 배치
    for item in pair_classes:
        if len(val_pairs) >= target_val:
            break
        cls_set = item[2]
        if any(not class_seen[c] for c in cls_set):
            val_pairs.append(item[:2])
            for c in cls_set:
                class_seen[c] = True

    # (2) 남은 자리 채우기
    for item in pair_classes:
        if len(val_pairs) >= target_val:
            break
        pair = item[:2]
        if pair not in val_pairs:
            val_pairs.append(pair)

    # (3) train은 나머지
    val_set = set(val_pairs)
    train_pairs = [(img, lbl) for (img, lbl, _) in pair_classes if (img, lbl) not in val_set]

print(f"[보정 후] train: {len(train_pairs)} | val: {len(val_pairs)}")

# ================================
# 5) 라벨 포맷/범위 간단 검증
# ================================
def check_label_row(line):
    parts = line.split()
    if len(parts) != 5:
        return False
    try:
        cid = int(float(parts[0]))
        bx = list(map(float, parts[1:5]))
        if cid < 0:
            return False
        return all(0.0 <= v <= 1.0 for v in bx)
    except:
        return False

bad_labels = []
for _, lbl in pairs:
    with open(lbl, "r", encoding="utf-8") as f:
        for li, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            if not check_label_row(line):
                bad_labels.append(f"{lbl}:{li}:{line}")
                break
if bad_labels:
    print(f"[경고] 포맷/범위가 이상한 라벨 파일 {len(bad_labels)}개 예) {bad_labels[:3]}")

# ================================
# 6) 최종 저장 (train/val 목록 + data.yaml) — 단 1회
# ================================
# (A) 최종 train/val 목록 저장
with open(train_list, "w", encoding="utf-8") as ft:
    for img, _ in train_pairs:
        ft.write(str(img).replace("\\", "/") + "\n")
with open(val_list, "w", encoding="utf-8") as fv:
    for img, _ in val_pairs:
        fv.write(str(img).replace("\\", "/") + "\n")

# (B) names 설정
# 기본 플레이스홀더
names = [f"class_{i}" for i in range(nc)]

# 실제 약품명 수동 오버라이드 (JSON에 없어도 무방)
# ※ 반드시 class id(0~nc-1) 인덱스가 맞아야 함
manual_names = {
    0: "보령부스파정 5mg",
    1: "동아가바펜틴정 800mg",
    2: "낙소졸정 500/20mg",
    3: "신바로정",
    4: "가바토파정 100mg",
    73: "넥시움정 40mg",  # ← 넥시움이 73번 클래스일 때
}
for k, v in manual_names.items():
    if 0 <= k < nc:
        names[k] = v

# (C) 넥시움 73번을 names로 쓸 계획이면 nc>=74 안전 체크 (필요 없으면 주석)
if 73 in manual_names:
    assert nc >= 74, f"넥시움(73번)을 names에 넣으려면 nc가 최소 74여야 합니다. 현재 nc={nc}"

# (D) data.yaml 저장 (최종 1회)
data_cfg = {
    "train": train_list.replace("\\", "/"),
    "val":   val_list.replace("\\", "/"),
    "nc":    nc,
    "names": names,
}
with open(DATA_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(data_cfg, f, allow_unicode=True, sort_keys=False)

print(" 최종 저장 완료")
print(f" - train 목록: {train_list}")
print(f" - val   목록: {val_list}")
print(f" - data.yaml: {DATA_YAML}")
print(f" - nc={nc}, 예시 names(0~5): {names[:6]}")


총 이미지: 1478 | 총 라벨: 1478 | 페어링 성공: 1478
추정 클래스 수 nc=74
[초기] train: 1331 | val: 147
[보정 후] train: 1331 | val: 147
✅ 최종 저장 완료
 - train 목록: E:\pytorch_env\YOLOv8x_dataset\_splits\train.txt
 - val   목록: E:\pytorch_env\YOLOv8x_dataset\_splits\val.txt
 - data.yaml: E:\pytorch_env\YOLOv8x_dataset\data.yaml
 - nc=74, 예시 names(0~5): ['보령부스파정 5mg', '동아가바펜틴정 800mg', '낙소졸정 500/20mg', '신바로정', '가바토파정 100mg', 'class_5']


---

## (1-B) 과적합 유도형 (리더보드 점수 극대화 버전)

### 목적

* **train 세트와 val 세트를 완전히 동일하게 구성**
* 검증에서도 학습 데이터를 그대로 사용해 **mAP 점수 극대화**


### 주요 특징

1. 전체 이미지/라벨을 **train=val 동일 세트로 구성** (VAL_RATIO 없음)
2. 라벨/이미지 페어링 및 포맷 검증은 동일하게 수행
3. 모든 데이터를 학습·검증 모두에 사용 → **의도적 과적합 상태**
4. `"넥시움정 40mg"` (73번 클래스) **names에서 제외됨**
5. 실제 일반화 성능 평가는 불가능하나, **리더보드 점수는 유리**

### 사용 시나리오

* **test 데이터가 train 데이터와 매우 유사할 때**
* **리더보드 점수 최적화**나 실험적 과적합 평가용

### 출력 파일

| 파일명                 | 설명                          |
| ------------------- | --------------------------- |
| `_splits/train.txt` | 전체 데이터 목록                   |
| `_splits/val.txt`   | **train과 동일한 목록**           |
| `data.yaml`         | train=val, 넥시움 제거(names 수정) |




In [ ]:
import os, glob, random, yaml
from pathlib import Path

# ================================
# 설정
# ================================
IMG_DIR = r"E:\pytorch_env\YOLOv8x_dataset\images"
LBL_DIR = r"E:\pytorch_env\YOLOv8x_dataset\labels"
SEED = 777
random.seed(SEED)

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

DATA_ROOT = str(Path(IMG_DIR).parent)
LIST_DIR = os.path.join(DATA_ROOT, "_splits")
os.makedirs(LIST_DIR, exist_ok=True)
train_list = os.path.join(LIST_DIR, "train.txt")
val_list   = os.path.join(LIST_DIR, "val.txt")  # ← 동일 파일로 덮어쓸 예정
DATA_YAML  = os.path.join(DATA_ROOT, "data.yaml")

def stem(p): return Path(p).stem

# ================================
# 1) 이미지-라벨 페어링
# ================================
images = [p for ext in IMG_EXTS for p in glob.glob(os.path.join(IMG_DIR, f"**/*{ext}"), recursive=True)]
images = sorted(set(images))
labels = glob.glob(os.path.join(LBL_DIR, "**/*.txt"), recursive=True)
label_map = {stem(p): p for p in labels}

pairs, miss_lbl, miss_img = [], [], []
for img in images:
    s = stem(img)
    if s in label_map: pairs.append((img, label_map[s]))
    else: miss_lbl.append(img)

for s, p in label_map.items():
    found = any(os.path.exists(os.path.join(Path(IMG_DIR), *(Path(p).parts[len(Path(LBL_DIR).parts):-1]), s + e)) for e in IMG_EXTS)
    if not found and not any(stem(i) == s for i in images):
        miss_img.append(p)

print(f"총 이미지: {len(images)} | 총 라벨: {len(labels)} | 페어링 성공: {len(pairs)}")
if miss_lbl: print(f"[경고] 라벨 없는 이미지 {len(miss_lbl)}개 (학습에서 제외): 예) {miss_lbl[:3]}")
if miss_img: print(f"[경고] 이미지 없는 라벨 {len(miss_img)}개 (학습에서 제외): 예) {miss_img[:3]}")
assert len(pairs) > 0, "이미지-라벨 페어가 없습니다."

# ================================
# 2) 클래스 수 자동 추출 (라벨 최대 class id + 1)
# ================================
def max_class_id_from_label(txt_path):
    mx = -1
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip(): continue
            try:
                cid = int(float(line.split()[0]))
                mx = max(mx, cid)
            except:
                pass
    return mx

global_max = -1
for _, lbl in pairs:
    m = max_class_id_from_label(lbl)
    global_max = max(global_max, m)
assert global_max >= 0, "라벨에서 클래스 id를 찾지 못했습니다."
nc = global_max + 1
print(f"추정 클래스 수 nc={nc}")

# ================================
# 3) 전량을 train으로 사용 (val=동일 세트)
# ================================
train_pairs = pairs[:]           # 전체를 학습에 사용
val_pairs   = pairs[:]           # 검증도 같은 세트(=과적합 점수 의도)

print(f"[세트 구성] train: {len(train_pairs)} | val(=train): {len(val_pairs)}")

# ================================
# 4) 라벨 포맷 간단 검증
# ================================
def check_label_row(line):
    p = line.split()
    if len(p) != 5: return False
    try:
        cid = int(float(p[0]))
        bx  = list(map(float, p[1:5]))
        if cid < 0: return False
        return all(0.0 <= v <= 1.0 for v in bx)
    except:
        return False

bad_labels = []
for _, lbl in pairs:
    with open(lbl, "r", encoding="utf-8") as f:
        for li, line in enumerate(f, 1):
            if not line.strip(): continue
            if not check_label_row(line):
                bad_labels.append(f"{lbl}:{li}:{line}")
                break
if bad_labels:
    print(f"[경고] 포맷/범위 이상 라벨 {len(bad_labels)}개 예) {bad_labels[:3]}")

# ================================
# 5) 목록/데이터 설정 저장
# ================================
with open(train_list, "w", encoding="utf-8") as ft:
    for img, _ in train_pairs: ft.write(str(img).replace("\\", "/") + "\n")
# val도 동일 파일을 그냥 복사 저장
with open(val_list, "w", encoding="utf-8") as fv:
    for img, _ in val_pairs: fv.write(str(img).replace("\\", "/") + "\n")

# names: 넥시움 제거했으니 수동 목록에서 73번은 절대 넣지 마세요!
names = [f"class_{i}" for i in range(nc)]
manual_names = {
    0: "보령부스파정 5mg",
    1: "동아가바펜틴정 800mg",
    2: "낙소졸정 500/20mg",
    3: "신바로정",
    4: "가바토파정 100mg",
    # 73: "넥시움정 40mg",  # 제거
}
for k, v in manual_names.items():
    if 0 <= k < nc: names[k] = v

data_cfg = {
    "train": train_list.replace("\\", "/"),
    "val":   val_list.replace("\\", "/"),   # ← 검증=학습세트
    "nc":    nc,
    "names": names,
}
with open(DATA_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(data_cfg, f, allow_unicode=True, sort_keys=False)

print(" 최종 저장 완료")
print(f" - train 목록: {train_list}")
print(f" - val   목록: {val_list} (train과 동일)")
print(f" - data.yaml: {DATA_YAML}")
print(f" - nc={nc}, 예시 names(0~5): {names[:6]}")


총 이미지: 1472 | 총 라벨: 1472 | 페어링 성공: 1472
추정 클래스 수 nc=73
[세트 구성] train: 1472 | val(=train): 1472
✅ 최종 저장 완료
 - train 목록: E:\pytorch_env\YOLOv8x_dataset\_splits\train.txt
 - val   목록: E:\pytorch_env\YOLOv8x_dataset\_splits\val.txt (train과 동일)
 - data.yaml: E:\pytorch_env\YOLOv8x_dataset\data.yaml
 - nc=73, 예시 names(0~5): ['보령부스파정 5mg', '동아가바펜틴정 800mg', '낙소졸정 500/20mg', '신바로정', '가바토파정 100mg', 'class_5']


---

## (2) data.yaml 내용 확인
- YOLO 학습 설정 파일(`data.yaml`)을 불러와 클래스 개수(`nc`)와 특정 클래스 이름을 출력.  
- `names[73]`이 존재할 경우, 인덱스 73번 클래스(예: "넥시움정 40mg")가 올바르게 등록되었는지 검증함.  
- 즉, 데이터 구성 단계에서 클래스 인덱스와 이름 매핑이 정상인지 빠르게 확인하는 테스트용 코드.


In [24]:
import yaml, os
DATA_YAML = r"E:\pytorch_env\YOLOv8x_dataset\data.yaml"
with open(DATA_YAML, "r", encoding="utf-8") as f:
    y = yaml.safe_load(f)
print("nc:", y["nc"], "names[73]:", y["names"][73] if y["nc"]>73 else None)
# 기대: nc: 74, names[73] == "넥시움정 40mg"


nc: 73 names[73]: None


---

## (3) 검증 세트 클래스 분포 확인
- 검증 이미지 목록(`val.txt`)을 기준으로 각 라벨 파일을 열어 클래스 ID를 카운트함.  
- `collections.Counter`를 이용해 클래스별 등장 횟수를 집계하고, 상위 10개 클래스를 출력.  
- 검증 세트(`val`)에 모든 클래스가 균형 있게 포함되어 있는지 빠르게 확인하기 위한 코드.


In [25]:
from collections import Counter
import os
VAL_LIST = r"E:\pytorch_env\YOLOv8x_dataset\_splits\val.txt"
LBL_DIR  = r"E:\pytorch_env\YOLOv8x_dataset\labels"
cnt = Counter()
with open(VAL_LIST, "r", encoding="utf-8") as f:
    for p in f:
        stem = os.path.splitext(os.path.basename(p.strip()))[0]
        with open(os.path.join(LBL_DIR, stem + ".txt"), "r", encoding="utf-8") as lf:
            for ln in lf:
                cid = int(float(ln.split()[0]))
                cnt[cid]+=1
print("val 클래스 수:", len(cnt), "예시:", cnt.most_common(10))


val 클래스 수: 73 예시: [(50, 619), (12, 248), (37, 199), (1, 175), (34, 171), (29, 161), (2, 154), (45, 137), (17, 131), (55, 124)]



---

## (4-A) YOLOv8 두단계 학습 파이프라인

* **Ultralytics YOLO**를 사용해 알약 검출 모델을 학습하는 두단계 학습 스크립트.
* **Stage-A**: 640px 해상도에서 기본 학습(Base training) 수행.
* **Stage-B**: Stage-A의 best 가중치를 불러와 960px 해상도에서 고해상도 파인튜닝(Fine-tuning).
* 각 단계별 학습 인자(optimizer, augmentation, lr 등)를 명시적으로 설정하며,
  `best.pt` 파일을 자동 탐색 후 다음 단계에 전달함.


### Stage-A: 기본 학습 (Base Training, 640px)

* 사전 학습된 **`yolov8m.pt`**를 기반으로 640×640 해상도에서 **80 epoch** 학습.
* 데이터셋(`data.yaml`)을 사용하며, **색상 변환·확대·이동 등 다양한 데이터 증강** 적용.
* **SGD 옵티마이저**, cosine learning rate 스케줄, warmup 3 epoch 사용.
* `hsv_h/s/v`, `translate`, `scale`, `fliplr` 등의 augmentation 비율 명시.
* `Exp/pill_v8m_640/weights/best.pt` 위치에 best 가중치 저장.
* 학습 설정 주요 인자:

  * `imgsz=640`, `batch=32`, `epochs=80`
  * `lr0=0.01`, `lrf=0.1`, `weight_decay=0.0005`
  * `mosaic=0.2`, `mixup=0.0`, `copy_paste=0.0`

### Stage-B: 고해상도 파인튜닝 (Fine-Tuning, 960px)

* **Stage-A의 best 모델을 불러와** 960×960 해상도에서 추가 학습(40 epoch).
* **학습률 감소(lr0=0.003)** 및 **데이터 증강 축소**로 세밀한 fine-tuning 수행.
* Mosaic, Mixup 비활성화(`mosaic=0.0`, `mixup=0.0`)로 고해상도 안정성 확보.
* `Exp/pill_v8m_960_ft/weights/best.pt` 경로에 결과 저장.
* 주요 인자:

  * `imgsz=960`, `batch=16`, `epochs=40`
  * `lr0=0.003`, `lrf=0.05`
  * `close_mosaic=100`, `scale=0.2`

### 최종 검증 단계

* Stage-B에서 학습된 best 모델(`best.pt`)을 이용해 검증 수행.
* 입력 해상도 960, `conf=0.001`, `iou=0.70` 기준으로 평가.
* **mAP, Precision, Recall** 등 주요 지표를 산출하고 콘솔에 출력.
* 검증 결과는 JSON 파일 및 시각화(`plots=True`) 형태로 저장됨.


In [ ]:
from ultralytics import YOLO
import os, glob

# 경로 설정
DATA_YAML = r"E:\pytorch_env\YOLOv8x_dataset\data.yaml"
PROJECT   = r"E:\pytorch_env\ai05-level1-project\Exp"
EXP_A     = "pill_v8m_640"
EXP_B     = "pill_v8m_960_ft"

os.makedirs(PROJECT, exist_ok=True)

# Stage-A (base training at 640px)
model_a = YOLO("yolov8m.pt")

train_args_a = {
    "data": DATA_YAML,
    "imgsz": 640,
    "epochs": 80,
    "batch": 32,          
    "device": 0,
    "optimizer": "SGD",
    "lr0": 0.01, "lrf": 0.1,
    "warmup_epochs": 3,
    "cos_lr": True,
    "close_mosaic": 10,
    "hsv_h": 0.015, "hsv_s": 0.7, "hsv_v": 0.4,
    "degrees": 0.0, "translate": 0.08, "scale": 0.5, "shear": 0.0, "perspective": 0.0,
    "fliplr": 0.5, "flipud": 0.0,
    "mosaic": 0.2, "mixup": 0.0, "copy_paste": 0.0, "erasing": 0.0,
    "box": 7.0, "cls": 0.7, "dfl": 1.5,
    "weight_decay": 0.0005,
    "patience": 30,
    "cache": "ram",
    "workers": 4,
    "verbose": True,
    "project": PROJECT, "name": EXP_A,
}
results_a = model_a.train(**train_args_a)

# Stage-A best 모델 경로
model_a_best = getattr(model_a, "ckpt_path", None) or glob.glob(
    os.path.join(PROJECT, EXP_A, "weights", "best.pt")
)[0]
print(f" Best Stage-A model: {model_a_best}")

# Stage-B (fine-tune at 960px)
model_b = YOLO(model_a_best)

train_args_b = {
    "data": DATA_YAML,
    "imgsz": 960,
    "epochs": 40,
    "batch": 16,
    "device": 0,
    "optimizer": "SGD",
    "lr0": 0.003, "lrf": 0.05,
    "warmup_epochs": 2,
    "cos_lr": True,
    "close_mosaic": 100,
    "mosaic": 0.0, "mixup": 0.0, "scale": 0.2,
    "box": 7.5, "cls": 0.5, "dfl": 1.5,
    "weight_decay": 0.0005,
    "cache": "ram",
    "workers": 4,
    "verbose": True,
    "project": PROJECT, "name": EXP_B,
}
results_b = model_b.train(**train_args_b)

# Stage-B best 모델
model_b_best = getattr(model_b, "ckpt_path", None) or glob.glob(
    os.path.join(PROJECT, EXP_B, "weights", "best.pt")
)[0]
print(f" Best Stage-B model: {model_b_best}")

# 최종 검증
val_metrics = model_b.val(
    model=model_b_best,
    data=DATA_YAML,
    imgsz=960,
    conf=0.001,
    iou=0.70,
    plots=True,
    save_json=True,
)

try:
    print({k: float(v) for k, v in val_metrics.results_dict.items()})
except Exception as e:
    print("Validation metrics 출력 오류:", e)
    print(val_metrics)


Ultralytics 8.3.221  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.0, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.7, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=E:\pytorch_env\YOLOv8x_dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.2, multi_scale=False, name=pill_v8m_640, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=30, perspective


---

## (4-B) YOLOv8 과적합(Overfit) 실험 파이프라인

* **Ultralytics YOLOv8** 기반의 **의도적 과적합 실험(Overfitting Test)** 코드.
* 학습·검증 데이터가 동일(`train=val`)하도록 구성된 `data.yaml`을 사용하며,
  **데이터 증강(Augmentation)을 완전히 비활성화**하여 빠르게 수렴시키는 것이 목표.
* 고해상도(960px)에서 장시간(200 epoch) 학습하여,
  모델의 최대 표현력(capacity)을 검증함.


### 설정 개요

* **입력 해상도:** 960 × 960
* **모델:** `yolov8m.pt` (중형 YOLOv8)
* **프로젝트 경로:** `E:\pytorch_env\ai05-level1-project\Exp`
* **실험 이름:** `pill_v8m_960_overfit_v5`
* **데이터셋 설정:** `data.yaml` (train=val 동일 구성)

### 학습 파라미터 요약

| 항목              | 설정값          | 설명                             |
| --------------- | ------------ | ------------------------------ |
| `epochs`        | 200          | 장기 학습으로 과적합 유도                 |
| `batch`         | 24           | GPU 메모리 여유에 맞춘 중간 배치 크기        |
| `optimizer`     | SGD          | 안정적이고 직관적인 수렴                  |
| `lr0` / `lrf`   | 0.015 / 0.20 | 학습률 초기값 및 종료 비율                |
| `cos_lr`        | False        | Cosine 스케줄링 비활성화               |
| `weight_decay`  | 0.0          | 가중치 감쇠 제거 (과적합 유도)             |
| `warmup_epochs` | 1.0          | 빠른 초기 수렴 유도                    |
| `rect`          | False        | Shuffle 활성화로 mini-batch 다양성 확보 |
| `save_period`   | 80           | 80 epoch 단위로 체크포인트 저장          |


### 데이터 증강 (Augmentation) 설정

* 모든 형태의 증강을 **완전히 비활성화(OFF)** 하여,
  데이터셋 원본 그대로 학습하도록 구성.

```text
"hsv_h": 0.0, "hsv_s": 0.0, "hsv_v": 0.0
"degrees": 0.0, "translate": 0.0, "scale": 0.0
"shear": 0.0, "perspective": 0.0
"fliplr": 0.0, "flipud": 0.0
"mosaic": 0.0, "mixup": 0.0, "copy_paste": 0.0
```

이 설정은 일반화 성능보다는 **훈련 세트 완전 적합(100% fit)** 을 목표로 함.


### 학습 흐름 요약

1. **모델 초기화:**

   * `YOLO("yolov8m.pt")` 로 사전학습 가중치 불러옴.

2. **학습 실행:**

   * `model.train(**train_args)` 실행.
   * 지정된 `Exp/pill_v8m_960_overfit_v5/` 디렉토리에 로그 및 가중치 저장.

3. **best.pt 자동 탐색:**

   * 학습 완료 후 `weights/best.pt` 경로를 자동 탐색.

4. **평가 단계 (`model.val`)**

   * `data.yaml`의 `val`이 `train`과 동일하므로,
     이 검증은 **“학습 세트 재평가” (Overfit 상태 점검)** 를 의미함.
   * 주요 평가 기준:

     * `imgsz=960`, `conf=0.001`, `iou=0.70`
     * `plots=True`, `save_json=True` → 시각화 및 COCO-style 결과 저장


### 결과 해석

* 출력된 `val_metrics`는 실제 검증이 아닌 **학습 세트 기준의 mAP, precision, recall** 값.
* 값이 0.99 이상에 수렴할 경우, **모델이 데이터에 완전 적합**했다는 의미.
* 일반화 능력 검증에는 부적합하지만,
  **모델 용량(capacity)** 및 **라벨 품질 점검**에 유용함.


### 주요 출력 경로

| 경로                                            | 설명               |
| --------------------------------------------- | ---------------- |
| `Exp/{EXP_NAME}/weights/best.pt` | 최종 best 모델 가중치   |
| `Exp/{EXP_NAME}/results.csv`     | 학습 로그            |
| `Exp/{EXP_NAME}/val_batch*.jpg`  | 검증 시각화 결과        |
| `Exp/{EXP_NAME}/plots/`          | mAP/PR 곡선 시각화 폴더 |



In [ ]:
from ultralytics import YOLO
import os, glob

DATA_YAML = r"E:\pytorch_env\YOLOv8x_dataset\data.yaml"
PROJECT   = r"E:\pytorch_env\ai05-level1-project\Exp"
EXP_NAME  = "pill_v8m_960_overfit_v5"  # ← 여기 이름과 train_args["name"]을 반드시 일치

os.makedirs(PROJECT, exist_ok=True)

# 모델 선택 (여유되면 yolov8x.pt)
model = YOLO("yolov8m.pt")

train_args = {
    "data": DATA_YAML,
    "imgsz": 960,
    "epochs": 200,
    "batch": 24,
    "device": 0,

    # 과적합 유도(빠른 수렴)
    "optimizer": "SGD",
    "lr0": 0.015,
    "lrf": 0.20,
    "cos_lr": False,
    "warmup_epochs": 1.0,
    "weight_decay": 0.0,

    # 증강 완전 OFF
    "hsv_h": 0.0, "hsv_s": 0.0, "hsv_v": 0.0,
    "degrees": 0.0, "translate": 0.0, "scale": 0.0, "shear": 0.0, "perspective": 0.0,
    "fliplr": 0.0, "flipud": 0.0,
    "mosaic": 0.0, "mixup": 0.0, "copy_paste": 0.0,

    # 기타
    "box": 7.0, "cls": 0.7, "dfl": 1.5,
    "patience": 0,
    "cache": "ram",
    "workers": 4,
    "seed": 42,
    "rect": False,  # shuffle 유도
    "project": PROJECT,
    "name": EXP_NAME,  # ← EXP_NAME과 일치시킴
    "verbose": True,

    # (선택) 훈련 중 검증 끄기 원하면 주석 해제
    # "val": False,

    # (선택) 중간 체크포인트 보존
    "save_period": 80,
}

# === 울트라리틱스가 받지 않는 키들 안전 제거(지금은 없음, 그래도 방어 코드 유지) ===
for bad in ['albumentations', 'augmentations', 'auto_augment', 'erasing']:
    if bad in train_args:
        print(f" remove invalid arg: {bad}={train_args.pop(bad)}")

results = model.train(**train_args)

# best.pt 경로
best_pt = getattr(model, "ckpt_path", None) or glob.glob(
    os.path.join(PROJECT, EXP_NAME, "weights", "best.pt")
)[0]
print("Best model:", best_pt)

# 훈련 후 평가
# data.yaml에서 val을 train과 동일하게 만들어뒀다면, 아래 평가는 '학습 세트 평가'가 됨
val_metrics = model.val(
    model=best_pt,
    data=DATA_YAML,
    imgsz=960,
    conf=0.001,
    iou=0.70,
    plots=True,
    save_json=True,
)

try:
    print({k: float(v) for k, v in val_metrics.results_dict.items()})
except Exception as e:
    print("Validation metrics 출력 오류:", e)
    print(val_metrics)


Ultralytics 8.3.221  Python-3.11.14 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.0, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.7, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:\pytorch_env\YOLOv8x_dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.015, lrf=0.2, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=pill_v8m_960_overfit_v5, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=0, 

e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48372 (\N{HANGUL SYLLABLE BO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 47161 (\N{HANGUL SYLLABLE RYEONG}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48512 (\N{HANGUL SYLLABLE BU}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 49828 (\N{HANGUL SYLLABLE SEU}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) Dej

                   all       1472       5599      0.994      0.996      0.995      0.994
             5mg        119        119      0.998          1      0.995      0.995
          800mg        175        175      0.998          1      0.995      0.995
          500/20mg        142        154          1      0.787      0.995      0.995
                           48         48      0.994          1      0.995      0.995
            100mg         97         97      0.997          1      0.995      0.995
               class_5         41         41      0.993          1      0.995      0.995
               class_6        102        102          1      0.998      0.995      0.995
               class_7         60         60      0.995          1      0.995      0.995
               class_8         98         98      0.997          1      0.995      0.995
               class_9         64         64      0.996          1      0.995      0.995
              class_10         37         37   

e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48372 (\N{HANGUL SYLLABLE BO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 47161 (\N{HANGUL SYLLABLE RYEONG}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48512 (\N{HANGUL SYLLABLE BU}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 49828 (\N{HANGUL SYLLABLE SEU}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
e:\anaconda3\envs\codeit_project_env\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) Dej

                   all       1472       5599      0.994      0.996      0.995      0.994
             5mg        119        119      0.998          1      0.995      0.995
          800mg        175        175      0.998          1      0.995      0.995
          500/20mg        142        154          1      0.789      0.995      0.995
                           48         48      0.994          1      0.995      0.995
            100mg         97         97      0.997          1      0.995      0.995
               class_5         41         41      0.993          1      0.995      0.995
               class_6        102        102          1      0.997      0.995      0.995
               class_7         60         60      0.995          1      0.995      0.995
               class_8         98         98      0.997          1      0.995      0.995
               class_9         64         64      0.996          1      0.995      0.995
              class_10         37         37   


---

## (5) YOLOv8 예측(Inference) 

* 학습이 완료된 YOLOv8 모델(`best.pt`)을 불러와
  **테스트 이미지 폴더 전체에 대해 객체 감지(Pill Detection)** 를 수행하는 코드.
* 예측된 결과는 **이미지(`.jpg`) + 라벨(`.txt`)** 형태로 저장되며,
  `.txt` 파일은 후처리 시 `.csv`로 변환 가능함.


### 기본 설정

| 항목                             | 설명                                                                               |
| ------------------------------ | -------------------------------------------------------------------------------- |
| **모델 경로**                      | `E:\pytorch_env\ai05-level1-project\Exp\pill_v8m_960_overfit_v5\weights\best.pt` |
| **테스트 이미지 폴더**                 | `E:\pytorch_env\ai05-level1-project\test_images`                                 |
| **출력 경로 (project)**            | `E:\pytorch_env\ai05-level1-project\Predictions`                                 |
| **실험 이름 (name)**               | `pill_v8m_960_overfit_v5`                                                        |
| **입력 해상도(imgsz)**              | 960 × 960                                                                        |
| **Confidence threshold(conf)** | 0.25 (감지 민감도)                                                                    |
| **IoU threshold(iou)**         | 0.5                                                                              |
| **저장 옵션**                      | `save=True`, `save_txt=True` → 결과 이미지 + 라벨 텍스트 동시 저장                             |


### 코드 동작 순서

1. **모델 로드**

   ```python
   model = YOLO(model_path)
   ```

   * 학습 완료된 YOLOv8 모델(`best.pt`) 불러오기
   * 구조와 가중치가 자동으로 초기화됨

2. **테스트 이미지 폴더 지정**

   ```python
   test_dir = r"E:\pytorch_env\ai05-level1-project\test_images"
   ```

   * 해당 폴더 내 모든 `.jpg`, `.png` 파일을 자동 탐색

3. **예측 실행**

   ```python
   results = model.predict(
       source=test_dir,
       imgsz=960,
       conf=0.25,
       iou=0.5,
       save=True,
       save_txt=True,
       project=project_dir,
       name=exp_name
   )
   ```

   * YOLO의 `predict()` 메서드로 폴더 단위 예측 수행
   * **감지 결과를 이미지와 라벨(txt)** 형태로 함께 저장
   * `save_txt=True`를 반드시 활성화해야 추후 CSV 변환 가능

4. **결과 경로 출력**

   ```python
   print(f"예측 완료! 결과는 {project_dir}\\{exp_name} 안에 저장되었습니다.")
   ```

   * 최종 결과 저장 위치를 콘솔에 표시
   * 예측이 끝난 후 `Predictions/pill_v8m_960_overfit_v5` 폴더가 생성됨


### 결과 저장 구조

```
E:\pytorch_env\ai05-level1-project\
└── Predictions\
    └── pill_v8m_960_overfit_v5\
        ├── image_001.jpg       # 감지 결과 이미지
        ├── image_001.txt       # YOLO 포맷 라벨 (class x_center y_center w h)
        ├── image_002.jpg
        └── image_002.txt
```


### 활용

* `conf` 값을 낮추면 **민감도(감지 수)** ↑,
  높이면 **정확도(노이즈 억제)** ↑.
* `save_txt=True` 덕분에 라벨 파일을 쉽게 `.csv`로 변환해
  **리더보드 제출용 데이터셋**으로 활용 가능.
* 다른 버전 모델을 테스트할 땐
  `exp_name`만 변경하면 구조를 그대로 재활용 가능 (`pill_v8m_960_overfit_v6` 등).



In [ ]:
from ultralytics import YOLO

# 학습 완료된 모델 로드
model_path = r"E:\pytorch_env\ai05-level1-project\Exp\pill_v8m_960_overfit_v5\weights\best.pt"
model = YOLO(model_path)

# 테스트 이미지 폴더
test_dir = r"E:\pytorch_env\ai05-level1-project\test_images"

# 예측 실행
project_dir = r"E:\pytorch_env\ai05-level1-project\Predictions"
exp_name = "pill_v8m_960_overfit_v5"  # 학습버전 변수로 따로 분리

results = model.predict(
    source=test_dir,
    imgsz=960,          
    conf=0.25,          
    iou=0.5,
    save=True,
    save_txt=True,       
    project=project_dir,
    name=exp_name
)

print(f"예측 완료! 결과는 {project_dir}\\{exp_name} 안에 저장되었습니다.")



image 1/843 E:\pytorch_env\ai05-level1-project\test_images\1.png: 960x736 1  5mg, 1  800mg, 1 class_7, 1 class_12, 50.1ms
image 2/843 E:\pytorch_env\ai05-level1-project\test_images\10.png: 960x736 1  5mg, 1  100mg, 1 class_9, 1 class_11, 7.1ms
image 3/843 E:\pytorch_env\ai05-level1-project\test_images\100.png: 960x736 1  5mg, 1 , 1  100mg, 1 class_5, 11.1ms
image 4/843 E:\pytorch_env\ai05-level1-project\test_images\1003.png: 960x736 1 class_31, 1 class_50, 1 class_56, 1 class_61, 6.5ms
image 5/843 E:\pytorch_env\ai05-level1-project\test_images\1004.png: 960x736 1 class_31, 1 class_50, 1 class_56, 1 class_61, 6.6ms
image 6/843 E:\pytorch_env\ai05-level1-project\test_images\1005.png: 960x736 1 class_31, 1 class_50, 1 class_56, 1 class_61, 10.3ms
image 7/843 E:\pytorch_env\ai05-level1-project\test_images\1006.png: 960x736 1 class_34, 1 class_50, 1 class_58, 1 class_59, 6.6ms
image 8/843 E:\pytorch_env\ai05-level1-project\test_images\1007.png: 960x736 1 class_34, 1 class_50, 1 class_58, 1

KeyboardInterrupt: 


---

##  (6-A) 제출용 CSV 생성 *넥시움정 40mg 특수처리 제거버전* (TTA + 매핑 추정 + NMS + 로컬 mAP)

### 목적

* YOLOv8 모델로 **테스트셋 TTA 예측** → **CSV 제출 파일** 생성
* **YOLO class → COCO category_id** 매핑을 **자동 추정(IoU 다수결)**
* (옵션) `sample_submission.csv` 기반 **allowlist** 적용
* **간단 NMS** 후 정렬/ID 재부여
* (옵션) **로컬 mAP@[0.75:0.95]** 평가


### 경로/입출력 요약

| 항목         | 경로/설정                                                        |
| ---------- | ------------------------------------------------------------ |
| 모델         | `Exp/pill_v8m_960_overfit_v5/weights/best.pt`                |
| 테스트 이미지 폴더 | `test_images/`                                               |
| 제출 CSV 출력  | `submission/submission_tta_conf_0.25.csv`                    |
| 데이터 루트     | `YOLOv8x_dataset/` (`images/`, `labels/`, `_splits/val.txt`) |
| (선택) 메타    | `test_meta.csv` (`file_name`, `image_id`)                    |
| (선택) 샘플 제출 | `sample_submission.csv` (allowlist)                          |


### 처리 흐름

1. **YOLO → JSON 매핑 자동 추정**

   * `train_annotations/*.json`(COCO)과 `labels/*.txt`(YOLO)를 **이미지 단위로 페어링**
   * **IoU(≥0.35)** 가 가장 큰 GT의 `category_id`를 해당 YOLO class의 표본 표결로 누적
   * **다수결**로 `yolo_id_to_original_id` 완성 (`MIN_VOTES=2`)
   * *매핑 실패 시 예측 박스는 보수적으로 **스킵*** (안전)

2. **TTA 예측 (test)**

   * `augment=True`, `conf=0.01`로 **저확신까지 최대 수집**
   * (선택) `test_meta.csv` 있으면 `image_id`를 정확 매핑, 없으면 파일명 숫자 파싱

3. **후처리**

   * `score >= FINAL_CONF_THRESHOLD(0.25)` 필터
   * (선택) `sample_submission.csv`의 카테고리만 **allowlist**
   * **간단 NMS**: (image_id, category_id) 그룹별 IoU 0.5 억제
   * 정렬(`image_id`, `score desc`) → `annotation_id` 재부여

4. **CSV 저장**

   * 컬럼: `annotation_id,image_id,category_id,bbox_x,bbox_y,bbox_w,bbox_h,score`

5. **로컬 mAP 평가 (옵션)**

   * `_splits/val.txt`에 명시된 이미지들에 대해 **mAP@[0.75:0.95]** 계산
   * `augment={False,True}` 두 설정으로 비교 *(train=val이면 과적합 점수)*

### 주요 파라미터

* 매핑 IoU 임계치: `IOU_THRESH=0.35`
* 다수결 최소 표본: `MIN_VOTES=2`
* 제출 필터: `FINAL_CONF_THRESHOLD=0.25`
* NMS: 그룹별 IoU 0.5

### 비고

* 매핑 실패분은 **제거(continue)** 하므로, **리콜이 떨어질 수 있음** → 필요 시 “미매핑 클래스 패스스루(cat=int(cls_id))”로 바꿔 실험 가능
* 간단 NMS는 **중복 감지 억제**에 효과적
* 로컬 mAP은 **상대 비교 지표**로 활용 (리더보드와 절대치는 다를 수 있음)


In [ ]:
# --- YOLOv8 TTA + 후처리 + 제출 CSV 생성 + 로컬 mAP@[0.75:0.95] 평가 (Nexium 특수처리 제거판) ---
from ultralytics import YOLO
import os, json, pandas as pd
import numpy as np
from PIL import Image
import torch
from torchmetrics.detection import MeanAveragePrecision
import glob

# -----------------------------
# 경로 설정
# -----------------------------
BASE_DIR       = r"E:\pytorch_env\ai05-level1-project"
EXP_DIR        = os.path.join(BASE_DIR, "Exp")
MODEL_PATH     = os.path.join(EXP_DIR, "pill_v8m_960_overfit_v5", "weights", "best.pt")  # ← 실험명 확인
TEST_IMG_DIR   = os.path.join(BASE_DIR, "test_images")
SUBMIT_DIR     = os.path.join(BASE_DIR, "submission")
os.makedirs(SUBMIT_DIR, exist_ok=True)
DATASET_ROOT   = r"E:\pytorch_env\YOLOv8x_dataset"
VAL_LIST_PATH  = os.path.join(DATASET_ROOT, "_splits", "val.txt")
LABELS_ROOT    = os.path.join(DATASET_ROOT, "labels")
IMAGES_ROOT    = os.path.join(DATASET_ROOT, "images")
FINAL_CONF_THRESHOLD = 0.25
SUBMIT_PATH    = os.path.join(SUBMIT_DIR, f"submission_tta_conf_{FINAL_CONF_THRESHOLD}.csv")

# (선택) 대회에서 제공하는 sample_submission.csv가 있다면 카테고리 allowlist로 활용
SAMPLE_SUB = os.path.join(BASE_DIR, "sample_submission.csv")

# (선택) 파일명→image_id 매핑 메타 (파일명 숫자 파싱 대신 정확 매핑용)
TEST_META = os.path.join(BASE_DIR, "test_meta.csv")

# -----------------------------
# 유틸
# -----------------------------
def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2 - ix1), max(0, iy2 - iy1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = max(0, (ax2 - ax1)) * max(0, (ay2 - ay1))
    area_b = max(0, (bx2 - bx1)) * max(0, (by2 - by1))
    union = area_a + area_b - inter
    return inter / max(union, 1e-9)

def load_yolo_boxes(txt_path, w_img, h_img):
    boxes, clses = [], []
    if not os.path.exists(txt_path): return boxes, clses
    with open(txt_path, "r", encoding="utf-8") as f:
        for ln in f:
            ln = ln.strip()
            if not ln: continue
            p = ln.split()
            cls_id = int(float(p[0]))
            xc, yc, w, h = map(float, p[1:5])
            xc *= w_img; yc *= h_img
            w  *= w_img; h  *= h_img
            x1 = xc - w/2; y1 = yc - h/2
            x2 = xc + w/2; y2 = yc + h/2
            boxes.append([x1, y1, x2, y2])
            clses.append(cls_id)
    return boxes, clses

# -----------------------------
#  YOLO class → JSON category_id 자동 매핑 (IoU 다수결)
# -----------------------------
TRAIN_ANN_DIR = os.path.join(BASE_DIR, "train_annotations")
coco_items = []  # (img_path, yolo_txt, coco_boxes_xyxy, coco_cids)

for jf in glob.glob(os.path.join(TRAIN_ANN_DIR, "**", "*.json"), recursive=True):
    try:
        with open(jf, "r", encoding="utf-8") as f:
            data = json.load(f)
        id2name = {img["id"]: img["file_name"] for img in data.get("images", [])}
        anns_by_img = {}
        for ann in data.get("annotations", []):
            anns_by_img.setdefault(ann["image_id"], []).append({
                "box": [ann["bbox"][0], ann["bbox"][1], ann["bbox"][0]+ann["bbox"][2], ann["bbox"][1]+ann["bbox"][3]],
                "cid": int(ann["category_id"]),
            })
        for img_id, fname in id2name.items():
            img_path = os.path.join(IMAGES_ROOT, fname)
            stem = os.path.splitext(os.path.basename(fname))[0]
            yolo_txt = os.path.join(LABELS_ROOT, stem + ".txt")
            if os.path.exists(img_path) and os.path.exists(yolo_txt):
                coco_boxes = [a["box"] for a in anns_by_img.get(img_id, [])]
                coco_cids  = [a["cid"] for a in anns_by_img.get(img_id, [])]
                if coco_boxes:
                    coco_items.append((img_path, yolo_txt, coco_boxes, coco_cids))
    except Exception as e:
        print("⚠️ COCO JSON 파싱 오류:", jf, e)

print(f"COCO-라벨 페어 수(매핑 후보): {len(coco_items)}")

from collections import defaultdict, Counter
pairs_votes = defaultdict(list)   # yolo_cls -> [matched_cid, ...]
SAMPLE_LIMIT = 10000
IOU_THRESH   = 0.35

count = 0
for img_path, yolo_txt, coco_boxes, coco_cids in coco_items:
    if count >= SAMPLE_LIMIT: break
    try:
        with Image.open(img_path) as im:
            w_img, h_img = im.size
    except:
        continue
    y_boxes, y_clses = load_yolo_boxes(yolo_txt, w_img, h_img)
    if not y_boxes: continue
    for yb, yc in zip(y_boxes, y_clses):
        best_iou, best_cid = 0.0, None
        for cb, cc in zip(coco_boxes, coco_cids):
            i = iou_xyxy(yb, cb)
            if i > best_iou:
                best_iou, best_cid = i, cc
        if best_cid is not None and best_iou >= IOU_THRESH:
            pairs_votes[yc].append(best_cid)
    count += 1

yolo_id_to_original_id = {}
MIN_VOTES = 2
for yc, votes in pairs_votes.items():
    if len(votes) >= MIN_VOTES:
        cid, _ = Counter(votes).most_common(1)[0]
        yolo_id_to_original_id[yc] = cid

print("추정된 매핑 개수:", len(yolo_id_to_original_id))
print(" YOLO → JSON category_id 매핑 예시 (0~9):")
for i in range(10):
    print(f"YOLO class {i} → category_id {yolo_id_to_original_id.get(i)}")

# -----------------------------
# 모델 로드
# -----------------------------
assert os.path.exists(MODEL_PATH), f"모델이 없습니다: {MODEL_PATH}"
model = YOLO(MODEL_PATH)

# -----------------------------
# (A) 테스트셋 → TTA 예측 → 제출 CSV 생성
# -----------------------------
test_images = sorted([f for f in os.listdir(TEST_IMG_DIR)
                      if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))])
assert len(test_images) > 0, f"테스트 이미지가 없습니다: {TEST_IMG_DIR}"
print(f" 테스트 이미지 수: {len(test_images)}")

# 파일명→image_id 메타 로드(있으면 정확 매핑)
name2id = None
if os.path.exists(TEST_META):
    try:
        meta = pd.read_csv(TEST_META)
        if {"file_name","image_id"}.issubset(meta.columns):
            name2id = dict(zip(meta["file_name"], meta["image_id"]))
            print(f"테스트 메타 로드 완료: {len(name2id)}개 매핑")
    except Exception as e:
        print(" TEST_META 로드 실패:", e)

# (선택) allowlist: sample_submission 에 있는 category_id만 허용
allow_cids = None
if os.path.exists(SAMPLE_SUB):
    try:
        ss = pd.read_csv(SAMPLE_SUB)
        if "category_id" in ss.columns:
            allow_cids = set(ss["category_id"].unique().tolist())
            print(f"[ALLOWLIST] sample_submission 기반 category_id 개수: {len(allow_cids)}")
    except Exception as e:
        print(" sample_submission 로드 실패:", e)

submission_rows = []

print("🔎 [SUBMIT] TTA 예측 실행 (augment=True, conf 낮춰 최대한 수집)")
for img_file in test_images:
    img_path = os.path.join(TEST_IMG_DIR, img_file)

    if name2id is not None and img_file in name2id:
        image_id = int(name2id[img_file])
    else:
        image_id = int(''.join(filter(str.isdigit, os.path.splitext(img_file)[0])) or 0)

    results = model.predict(
        img_path,
        imgsz=960,
        conf=0.01,      # 수집 극대화
        iou=0.45,
        device=0,
        augment=True,
        verbose=False
    )

    for r in results:
        b = r.boxes
        if b is None or len(b) == 0: continue
        xyxy = b.xyxy.cpu().numpy()
        confs = b.conf.cpu().numpy()
        clses = b.cls.cpu().numpy().astype(int)

        for (x1, y1, x2, y2), conf, cls_id in zip(xyxy, confs, clses):
            cat = yolo_id_to_original_id.get(int(cls_id), None)
            if cat is None:
                # 매핑 실패시: 보수적으로 스킵(또는 cat=int(cls_id)로 패스스루)
                continue
            submission_rows.append({
                "annotation_id": 0,  # 임시
                "image_id": image_id,
                "category_id": int(cat),
                "bbox_x": int(round(x1)),
                "bbox_y": int(round(y1)),
                "bbox_w": int(round(x2 - x1)),
                "bbox_h": int(round(y2 - y1)),
                "score": round(float(conf), 4)
            })

# DataFrame화 & 1차 필터
df = pd.DataFrame(submission_rows)
if df.empty:
    print(" [SUBMIT] 예측된 박스가 없습니다.")
else:
    print(f"[SUBMIT] 총 수집 박스: {len(df)}")
    df = df[df["score"] >= FINAL_CONF_THRESHOLD].copy()
    print(f"[SUBMIT] score >= {FINAL_CONF_THRESHOLD} 필터 후: {len(df)}")

    # (선택) allowlist 적용
    if allow_cids is not None:
        before = len(df)
        df = df[df["category_id"].isin(allow_cids)].copy()
        print(f"[ALLOWLIST] 카테고리 필터: {before} → {len(df)}")

    # 간단 NMS (image_id, category 별로)
    def nms_on_df(df_grp, iou_thr=0.5):
        if len(df_grp) <= 1: return df_grp
        boxes = df_grp[["bbox_x","bbox_y","bbox_w","bbox_h"]].to_numpy().astype(float)
        boxes[:,2] += boxes[:,0]  # x2
        boxes[:,3] += boxes[:,1]  # y2
        scores = df_grp["score"].to_numpy().astype(float)

        idxs = scores.argsort()[::-1]
        keep = []
        while idxs.size > 0:
            i = idxs[0]
            keep.append(i)
            if idxs.size == 1: break
            rest = idxs[1:]
            xx1 = np.maximum(boxes[i,0], boxes[rest,0])
            yy1 = np.maximum(boxes[i,1], boxes[rest,1])
            xx2 = np.minimum(boxes[i,2], boxes[rest,2])
            yy2 = np.minimum(boxes[i,3], boxes[rest,3])
            w = np.maximum(0.0, xx2 - xx1)
            h = np.maximum(0.0, yy2 - yy1)
            inter = w * h
            area_i = (boxes[i,2]-boxes[i,0]) * (boxes[i,3]-boxes[i,1])
            area_r = (boxes[rest,2]-boxes[rest,0]) * (boxes[rest,3]-boxes[rest,1])
            iou = inter / np.maximum(area_i + area_r - inter, 1e-9)
            idxs = rest[iou < iou_thr]
        return df_grp.iloc[keep]

    if not df.empty:
        before_nms = len(df)
        df = (df
              .groupby(["image_id","category_id"], group_keys=False)
              .apply(lambda g: nms_on_df(g, iou_thr=0.5)))
        print(f"[SUBMIT] 간단 NMS 적용: {before_nms} → {len(df)}")

    # 마무리 정렬/annotation_id 재부여
    df.sort_values(by=["image_id","score"], ascending=[True, False], inplace=True)
    df["annotation_id"] = range(1, len(df) + 1)
    df = df[["annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"]]

    # 저장
    df.to_csv(SUBMIT_PATH, index=False, encoding="utf-8-sig")
    print(f" Saved CSV: {SUBMIT_PATH}")
    print("unique category_id (sample):", df["category_id"].unique()[:12])

# -----------------------------
# (B) 로컬 평가: mAP@[0.75:0.95] (val=train이면 과적합 점수임)
# -----------------------------
def _yolo_txt_to_xyxy_pixels(lbl_path: str, img_w: int, img_h: int):
    boxes, labels = [], []
    if not os.path.exists(lbl_path): return boxes, labels
    with open(lbl_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5: continue
            cid = int(float(parts[0]))
            xc, yc, w, h = map(float, parts[1:5])
            xc *= img_w; yc *= img_h
            w  *= img_w; h  *= img_h
            x1 = xc - w/2; y1 = yc - h/2
            x2 = xc + w/2; y2 = yc + h/2
            boxes.append([x1, y1, x2, y2])
            labels.append(cid)
    return boxes, labels

def evaluate_map075_095(model, val_list_path, labels_root, use_tta=False):
    assert os.path.exists(val_list_path), f"val 리스트가 없습니다: {val_list_path}"
    with open(val_list_path, "r", encoding="utf-8") as f:
        val_imgs = [ln.strip() for ln in f if ln.strip()]

    iou_thresholds = np.round(np.arange(0.75, 1.0, 0.05), 2).tolist()
    metric = MeanAveragePrecision(iou_type="bbox", iou_thresholds=iou_thresholds)

    for img_path in val_imgs:
        try:
            with Image.open(img_path) as im:
                w_img, h_img = im.size
        except Exception:
            candidate = os.path.join(IMAGES_ROOT, os.path.basename(img_path))
            with Image.open(candidate) as im:
                w_img, h_img = im.size
            img_path = candidate

        preds_ultra = model.predict(
            img_path, imgsz=960, conf=0.001, iou=0.6,
            device=0, augment=use_tta, verbose=False
        )

        pred_items = []
        for r in preds_ultra:
            b = r.boxes
            if b is None or len(b) == 0:
                pred_items.append({"boxes": torch.zeros((0,4)), "scores": torch.zeros((0,)), "labels": torch.zeros((0,), dtype=torch.int64)})
                continue
            pred_items.append({
                "boxes": b.xyxy.cpu(),
                "scores": b.conf.cpu(),
                "labels": b.cls.cpu().to(torch.int64)
            })

        stem = os.path.splitext(os.path.basename(img_path))[0]
        gt_label_path = os.path.join(labels_root, stem + ".txt")
        gt_boxes, gt_labels = _yolo_txt_to_xyxy_pixels(gt_label_path, w_img, h_img)

        if len(gt_boxes) == 0:
            target_items = [{"boxes": torch.zeros((0,4)), "labels": torch.zeros((0,), dtype=torch.int64)}]
        else:
            target_items = [{"boxes": torch.tensor(gt_boxes, dtype=torch.float32), "labels": torch.tensor(gt_labels, dtype=torch.int64)}]

        metric.update(pred_items, target_items)

    res = metric.compute()
    return float(res["map"])

if os.path.exists(VAL_LIST_PATH):
    approx_map_notta = evaluate_map075_095(model, VAL_LIST_PATH, LABELS_ROOT, use_tta=False)
    print(" [LOCAL EVAL] Approx. mAP@[0.75:0.95] (NO TTA): {:.4f}".format(approx_map_notta))
    approx_map_tta = evaluate_map075_095(model, VAL_LIST_PATH, LABELS_ROOT, use_tta=True)
    print(" [LOCAL EVAL] Approx. mAP@[0.75:0.95] (with TTA): {:.4f}".format(approx_map_tta))
else:
    print(f"ℹ 로컬 평가 생략: VAL 리스트가 없습니다. ({VAL_LIST_PATH})")


COCO-라벨 페어 수(매핑 후보): 2381
추정된 매핑 개수: 73
✅ YOLO → JSON category_id 매핑 예시 (0~9):
YOLO class 0 → category_id 1899
YOLO class 1 → category_id 16550
YOLO class 2 → category_id 31704
YOLO class 3 → category_id 33008
YOLO class 4 → category_id 16547
YOLO class 5 → category_id 18109
YOLO class 6 → category_id 21025
YOLO class 7 → category_id 27925
YOLO class 8 → category_id 29344
YOLO class 9 → category_id 29450
🖼️ 테스트 이미지 수: 843
🔎 [SUBMIT] TTA 예측 실행 (augment=True, conf 낮춰 최대한 수집)
[SUBMIT] 총 수집 박스: 7935
[SUBMIT] score >= 0.25 필터 후: 4126
[SUBMIT] 간단 NMS 적용: 4126 → 4126
💾 Saved CSV: E:\pytorch_env\ai05-level1-project\submission\submission_tta_conf_0.25.csv
unique category_id (sample): [ 1899 24849 27732 27925 16550  6562 35205 29344 12246  3482 23202 19606]
📏 [LOCAL EVAL] Approx. mAP@[0.75:0.95] (NO TTA): 0.9950
📏 [LOCAL EVAL] Approx. mAP@[0.75:0.95] (with TTA): 0.4170


---

## (6-B) 제출용 CSV 생성 *넥시움정 40mg 고정 매핑버전* (TTA + 고정/추정 매핑 + 필터 + 로컬 mAP)

### 목적

* YOLOv8 모델로 **테스트셋 TTA 예측** → **CSV 제출 파일** 생성
* YOLO→JSON 매핑: **표결 추정 + 일부 클래스 고정 덮어쓰기** + **Nexium 고정**
* (선택) `test_meta.csv` 기반 안정적 image_id 매핑
* **정렬/annotation_id 재부여** (※ 기본 NMS는 빠져있음)
* (옵션) **로컬 mAP@[0.75:0.95]** 평가


### 경로/입출력 요약

| 항목         | 경로/설정                                                        |
| ---------- | ------------------------------------------------------------ |
| 모델         | `Exp/pill_v8m_960_overfit_v4/weights/best.pt`                |
| 테스트 이미지 폴더 | `test_images/`                                               |
| 제출 CSV 출력  | `submission/submission_tta_conf_0.25.csv`                    |
| 데이터 루트     | `YOLOv8x_dataset/` (`images/`, `labels/`, `_splits/val.txt`) |
| (선택) 메타    | `test_meta.csv` (`file_name`, `image_id`)                    |


### 처리 흐름

1. **매핑 생성 (강화판)**

   * COCO/YOLO 페어에서 **IoU≥0.35** 다수결로 1차 매핑
   * `fixed_head`로 **일부 클래스 확정 덮어쓰기**
   * **Nexium**: `YOLO_CLASS=73` → `category_id=10223` **고정**
   * 최종 `yolo_id_to_original_id` 확정

2. **TTA 예측 (test)**

   * `augment=True`, `conf=0.01`
   * `test_meta.csv` 있으면 **정밀 image_id 매핑** (없으면 숫자 파싱)

3. **후처리**

   * `score >= FINAL_CONF_THRESHOLD(0.25)` 필터
   * 정렬(`image_id`, `score desc`) → `annotation_id` 재부여
   * *(기본 코드에는 NMS 미포함 — 필요 시 #1의 NMS 함수 이식 권장)*

4. **CSV 저장**

   * 컬럼: `annotation_id,image_id,category_id,bbox_x,bbox_y,bbox_w,bbox_h,score`
   * (안전성 체크) `known_ids` 집합 바깥 `category_id`를 **이상치로 탐지**해 요약 출력

5. **로컬 mAP 평가 (옵션)**

   * `_splits/val.txt` 기준 **mAP@[0.75:0.95]** (TTA off/on 비교)


### 주요 파라미터

* 매핑 IoU 임계치: `IOU_THRESH=0.35`
* 다수결 최소 표본: `MIN_VOTES=2`
* 제출 필터: `FINAL_CONF_THRESHOLD=0.25`
* **Nexium 고정:** `73 → 10223`


### 주의/리뷰 포인트

* **중복 예측 루프**: 코드에 예측 루프가 **두 번** 들어가 있음(앞쪽 `name2id` 활용 루프 + 뒤쪽 숫자 파싱 루프).

  * 그대로 두면 **박스가 두 배로 쌓여 리콜/정밀도 왜곡** 및 파일 사이즈 증가.
  *  **앞쪽(메타 활용) 루프만 남기고 뒤쪽 루프는 제거** 권장.
* NMS가 **빠져있음** → 겹치는 박스가 많다면 #1의 **`nms_on_df` 함수**를 그대로 이식해서
  `df.groupby(["image_id","category_id"]).apply(nms_on_df)` 형태로 적용 추천.
* 알려진 매핑 집합(`known_ids`) 밖의 `category_id`가 **잔존**하면
  매핑 실패/노이즈일 확률 높음 → 임계점 조정 또는 매핑 보강 필요.

### 두 스크립트의 핵심 차이

* #1: **미매핑 박스는 스킵**, **NMS 포함**, allowlist 옵션 지원 → **보수적·정제형**
* #2: **고정 매핑(Nexium 포함)** + (기본) **NMS 없음** → **재현성·일관성 강화**, 다만 **중복 루프 제거 필요**




In [ ]:
# --- YOLOv8 TTA + 후처리 + 제출 CSV 생성 + 로컬 mAP@[0.75:0.95] 평가 ---
from ultralytics import YOLO
import os, json, pandas as pd
import numpy as np
from PIL import Image
import torch
from torchmetrics.detection import MeanAveragePrecision
import glob

# -----------------------------
# 경로 설정
# -----------------------------
BASE_DIR       = r"E:\pytorch_env\ai05-level1-project"
EXP_DIR        = os.path.join(BASE_DIR, "Exp")
MODEL_PATH     = os.path.join(EXP_DIR, "pill_v8m_960_overfit_v4", "weights", "best.pt")
TEST_IMG_DIR   = os.path.join(BASE_DIR, "test_images")
SUBMIT_DIR     = os.path.join(BASE_DIR, "submission")
os.makedirs(SUBMIT_DIR, exist_ok=True)
DATASET_ROOT   = r"E:\pytorch_env\YOLOv8x_dataset"
VAL_LIST_PATH  = os.path.join(DATASET_ROOT, "_splits", "val.txt")
LABELS_ROOT    = os.path.join(DATASET_ROOT, "labels")
IMAGES_ROOT    = os.path.join(DATASET_ROOT, "images")
FINAL_CONF_THRESHOLD = 0.25
SUBMIT_PATH    = os.path.join(SUBMIT_DIR, f"submission_tta_conf_{FINAL_CONF_THRESHOLD}.csv")

# -----------------------------
#  YOLO class → JSON category_id 자동 매핑 (IoU 기반) + 넥시움 고정
# -----------------------------
from collections import defaultdict, Counter, OrderedDict

TRAIN_ANN_DIR = os.path.join(BASE_DIR, "train_annotations")

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2 - ix1), max(0, iy2 - iy1)
    inter = iw * ih
    if inter <= 0:
        return 0.0
    area_a = max(0, (ax2 - ax1)) * max(0, (ay2 - ay1))
    area_b = max(0, (bx2 - bx1)) * max(0, (by2 - by1))
    union = area_a + area_b - inter
    return inter / max(union, 1e-9)

def load_yolo_boxes(txt_path, w_img, h_img):
    boxes, clses = [], []
    if not os.path.exists(txt_path):
        return boxes, clses
    with open(txt_path, "r", encoding="utf-8") as f:
        for ln in f:
            ln = ln.strip()
            if not ln:
                continue
            p = ln.split()
            cls_id = int(float(p[0]))
            xc, yc, w, h = map(float, p[1:5])
            xc *= w_img; yc *= h_img
            w  *= w_img; h  *= h_img
            x1 = xc - w/2; y1 = yc - h/2
            x2 = xc + w/2; y2 = yc + h/2
            boxes.append([x1, y1, x2, y2])
            clses.append(cls_id)
    return boxes, clses

# (A) COCO JSON에서 image/annotation 파싱 → 이미지와 라벨(txt) 페어 구성
coco_items = []  # list of (img_path, yolo_txt, coco_boxes_xyxy, coco_cids)
for jf in glob.glob(os.path.join(TRAIN_ANN_DIR, "**", "*.json"), recursive=True):
    try:
        with open(jf, "r", encoding="utf-8") as f:
            data = json.load(f)
        id2name = {img["id"]: img["file_name"] for img in data.get("images", [])}
        anns_by_img = defaultdict(list)
        for ann in data.get("annotations", []):
            x, y, w, h = ann["bbox"]
            anns_by_img[ann["image_id"]].append({
                "box": [x, y, x+w, y+h],
                "cid": int(ann["category_id"]),
            })
        for img_id, fname in id2name.items():
            img_path = os.path.join(IMAGES_ROOT, fname)
            stem = os.path.splitext(os.path.basename(fname))[0]
            yolo_txt = os.path.join(LABELS_ROOT, stem + ".txt")
            if os.path.exists(img_path) and os.path.exists(yolo_txt):
                coco_boxes = [a["box"] for a in anns_by_img.get(img_id, [])]
                coco_cids  = [a["cid"] for a in anns_by_img.get(img_id, [])]
                if coco_boxes:
                    coco_items.append((img_path, yolo_txt, coco_boxes, coco_cids))
    except Exception as e:
        print(" COCO JSON 파싱 오류:", jf, e)

print(f"COCO-라벨 페어 수(매핑 후보): {len(coco_items)}")

# (B) IoU로 yolo_cls ↔ coco_category_id 표본 매칭  [강화판]
pairs_votes = defaultdict(list)  # yolo_cls -> [matched_cid, ...]
SAMPLE_LIMIT = 10000             #  더 많이 스캔
IOU_THRESH   = 0.35              #  임계치 낮춤(0.5→0.35)

count = 0
for img_path, yolo_txt, coco_boxes, coco_cids in coco_items:
    if count >= SAMPLE_LIMIT:
        break
    try:
        with Image.open(img_path) as im:
            w_img, h_img = im.size
    except:
        continue
    y_boxes, y_clses = load_yolo_boxes(yolo_txt, w_img, h_img)
    if not y_boxes:
        continue

    # 간단 가속: coco_boxes가 많을 때는 대충 top-k 후보만 본다 (선택)
    # 여기선 그대로 전수 비교

    for yb, yc in zip(y_boxes, y_clses):
        best_iou, best_cid = 0.0, None
        for cb, cc in zip(coco_boxes, coco_cids):
            i = iou_xyxy(yb, cb)
            if i > best_iou:
                best_iou, best_cid = i, cc
        if best_cid is not None and best_iou >= IOU_THRESH:
            pairs_votes[yc].append(best_cid)
    count += 1

# (C) 다수결로 최종 매핑 생성 (+ 최소 샘플 수 보정)
yolo_id_to_original_id = {}
MIN_VOTES = 2  # 너무 적게 잡힌 건 노이즈일 수 있어 제외
for yc, votes in pairs_votes.items():
    if len(votes) >= MIN_VOTES:
        cid, n = Counter(votes).most_common(1)[0]
        yolo_id_to_original_id[yc] = cid

# (D) 우리가 이미 확정한 매핑 덮어쓰기 + 넥시움 고정
fixed_head = {
    0: 1899, 1: 16550, 2: 31704, 3: 33008, 4: 16547,
}
yolo_id_to_original_id.update(fixed_head)

NEXIUM_YOLO_CLASS = 73
NEXIUM_JSON_ID    = 10223
yolo_id_to_original_id[NEXIUM_YOLO_CLASS] = NEXIUM_JSON_ID

print("추정된 매핑 개수:", len(yolo_id_to_original_id))

print(" YOLO → JSON category_id 매핑 예시 (0~9):")
for i in range(10):
    print(f"YOLO class {i} → category_id {yolo_id_to_original_id.get(i)}")

# -----------------------------
# 모델 로드
# -----------------------------
assert os.path.exists(MODEL_PATH), f"모델이 없습니다: {MODEL_PATH}"
model = YOLO(MODEL_PATH)

# -----------------------------
# (A) 테스트셋 → TTA 예측 → 제출 CSV 생성
# -----------------------------
test_images = sorted([f for f in os.listdir(TEST_IMG_DIR)
                      if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp"))])
assert len(test_images) > 0, f"테스트 이미지가 없습니다: {TEST_IMG_DIR}"
print(f"🖼️ 테스트 이미지 수: {len(test_images)}")

submission_rows = []

print("🔎 [SUBMIT] TTA 예측 실행 (augment=True, conf 낮춰 최대한 수집)")

#  [여기 추가] image_id 안전 매핑용 메타 불러오기
TEST_META = os.path.join(BASE_DIR, "test_meta.csv")
name2id = None
if os.path.exists(TEST_META):
    meta = pd.read_csv(TEST_META)
    name2id = dict(zip(meta["file_name"], meta["image_id"]))
    print(f"테스트 메타 로드 완료: {len(name2id)}개 매핑")

# -------------------------------
# 기존 예측 루프 시작
# -------------------------------
for img_file in test_images:
    img_path = os.path.join(TEST_IMG_DIR, img_file)

    # 기존 코드 수정 부분
    if name2id is not None and img_file in name2id:
        image_id = int(name2id[img_file])  # 메타에 있으면 안전하게 매핑
    else:
        # fallback — 파일명에서 숫자 추출 (ex: image_123.jpg → 123)
        image_id = int(''.join(filter(str.isdigit, os.path.splitext(img_file)[0])) or 0)

    results = model.predict(
        img_path,
        imgsz=960,
        conf=0.01,
        iou=0.45,
        device=0,
        augment=True,
        verbose=False
    )
    ...


for img_file in test_images:
    img_path = os.path.join(TEST_IMG_DIR, img_file)
    image_id = int(''.join(filter(str.isdigit, os.path.splitext(img_file)[0])) or 0)

    results = model.predict(
        img_path,
        imgsz=960,
        conf=0.01,  # 낮은 conf로 최대 수집
        iou=0.45,
        device=0,
        augment=True,
        verbose=False
    )

    for r in results:
        b = r.boxes
        if b is None or len(b) == 0:
            continue
        xyxy = b.xyxy.cpu().numpy()
        confs = b.conf.cpu().numpy()
        clses = b.cls.cpu().numpy().astype(int)

        for (x1, y1, x2, y2), conf, cls_id in zip(xyxy, confs, clses):
            category_id = yolo_id_to_original_id.get(int(cls_id), int(cls_id))
            submission_rows.append({
                "annotation_id": 0,  # 임시 (아래에서 재부여)
                "image_id": image_id,
                "category_id": int(category_id),
                "bbox_x": int(round(x1)),
                "bbox_y": int(round(y1)),
                "bbox_w": int(round(x2 - x1)),
                "bbox_h": int(round(y2 - y1)),
                "score": round(float(conf), 4)
            })

# 후처리: conf threshold, 정렬, annotation_id 재부여
df = pd.DataFrame(submission_rows)
if df.empty:
    print(" [SUBMIT] 예측된 박스가 없습니다.")
else:
    print(f"[SUBMIT] 총 수집 박스: {len(df)}")
    df = df[df["score"] >= FINAL_CONF_THRESHOLD].copy()
    print(f"[SUBMIT] score >= {FINAL_CONF_THRESHOLD} 필터 후: {len(df)}")

    df.sort_values(by=["image_id", "score"], ascending=[True, False], inplace=True)
    df["annotation_id"] = range(1, len(df) + 1)
    df = df[["annotation_id", "image_id", "category_id", "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"]]

    # (검증) 카테고리 id 이상치 체크 + 안전 필터링
    known_ids = set(yolo_id_to_original_id.values()) | {NEXIUM_JSON_ID}
    # 매핑 실패의 전형적 패턴: category_id가 '자기 자신' 정수(예: 68, 72)로 남음
    bad_mask = ~df["category_id"].isin(known_ids)
    bad_count = int(bad_mask.sum())
        # 어떤 YOLO class가 필터에 걸렸는지 한눈에
    if bad_count:
        bad_classes = df.loc[bad_mask, "yolo_cls"] if "yolo_cls" in df.columns else None
        if bad_classes is not None:
            print("⚠️ 제거된 행의 YOLO class 카운트(상위):")
            print(bad_classes.value_counts().head(10))
    
    #  CSV 저장 추가
    os.makedirs(SUBMIT_DIR, exist_ok=True)
    df.to_csv(SUBMIT_PATH, index=False, encoding="utf-8-sig")
    print(f" Saved CSV: {SUBMIT_PATH}")


    # 참고: 어느 YOLO class에서 많이 발생했는지 간단 통계 (선택)
    # df에는 원래 cls가 없어서, 필요하면 위 predict 루프에서 원본 cls도 같이 저장하도록 확장 가능.


# -----------------------------
# (B) 로컬 평가: mAP@[0.75:0.95]
# -----------------------------
def _yolo_txt_to_xyxy_pixels(lbl_path: str, img_w: int, img_h: int):
    boxes, labels = [], []
    if not os.path.exists(lbl_path):
        return boxes, labels
    with open(lbl_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cid = int(float(parts[0]))
            xc, yc, w, h = map(float, parts[1:5])
            xc *= img_w; yc *= img_h
            w  *= img_w; h  *= img_h
            x1 = xc - w/2; y1 = yc - h/2
            x2 = xc + w/2; y2 = yc + h/2
            boxes.append([x1, y1, x2, y2])
            labels.append(cid)
    return boxes, labels

def evaluate_map075_095(model, val_list_path, labels_root, use_tta=False):
    assert os.path.exists(val_list_path), f"val 리스트가 없습니다: {val_list_path}"
    with open(val_list_path, "r", encoding="utf-8") as f:
        val_imgs = [ln.strip() for ln in f if ln.strip()]

    iou_thresholds = np.round(np.arange(0.75, 1.0, 0.05), 2).tolist()
    metric = MeanAveragePrecision(iou_type="bbox", iou_thresholds=iou_thresholds)

    for img_path in val_imgs:
        try:
            with Image.open(img_path) as im:
                w_img, h_img = im.size
        except Exception:
            candidate = os.path.join(IMAGES_ROOT, os.path.basename(img_path))
            with Image.open(candidate) as im:
                w_img, h_img = im.size
            img_path = candidate

        preds_ultra = model.predict(
            img_path, imgsz=960, conf=0.001, iou=0.6,
            device=0, augment=use_tta, verbose=False
        )

        pred_items = []
        for r in preds_ultra:
            b = r.boxes
            if b is None or len(b) == 0:
                pred_items.append({
                    "boxes": torch.zeros((0,4), dtype=torch.float32),
                    "scores": torch.zeros((0,), dtype=torch.float32),
                    "labels": torch.zeros((0,), dtype=torch.int64),
                })
                continue
            pred_items.append({
                "boxes": b.xyxy.cpu(),
                "scores": b.conf.cpu(),
                "labels": b.cls.cpu().to(torch.int64)
            })

        stem = os.path.splitext(os.path.basename(img_path))[0]
        gt_label_path = os.path.join(labels_root, stem + ".txt")
        gt_boxes, gt_labels = _yolo_txt_to_xyxy_pixels(gt_label_path, w_img, h_img)

        if len(gt_boxes) == 0:
            target_items = [{
                "boxes": torch.zeros((0,4), dtype=torch.float32),
                "labels": torch.zeros((0,), dtype=torch.int64),
            }]
        else:
            target_items = [{
                "boxes": torch.tensor(gt_boxes, dtype=torch.float32),
                "labels": torch.tensor(gt_labels, dtype=torch.int64),
            }]

        metric.update(pred_items, target_items)

    res = metric.compute()
    return float(res["map"])

if os.path.exists(VAL_LIST_PATH):
    approx_map_notta = evaluate_map075_095(model, VAL_LIST_PATH, LABELS_ROOT, use_tta=False)
    print(" [LOCAL EVAL] Approx. mAP@[0.75:0.95] (NO TTA): {:.4f}".format(approx_map_notta))
    approx_map_tta = evaluate_map075_095(model, VAL_LIST_PATH, LABELS_ROOT, use_tta=True)
    print(" [LOCAL EVAL] Approx. mAP@[0.75:0.95] (with TTA): {:.4f}".format(approx_map_tta))
else:
    print(f" 로컬 평가 생략: VAL 리스트가 없습니다. ({VAL_LIST_PATH})")


COCO-라벨 페어 수(매핑 후보): 2396
추정된 매핑 개수: 74
✅ YOLO → JSON category_id 매핑 예시 (0~9):
YOLO class 0 → category_id 1899
YOLO class 1 → category_id 16550
YOLO class 2 → category_id 31704
YOLO class 3 → category_id 33008
YOLO class 4 → category_id 16547
YOLO class 5 → category_id 18109
YOLO class 6 → category_id 21025
YOLO class 7 → category_id 27925
YOLO class 8 → category_id 29344
YOLO class 9 → category_id 29450
🖼️ 테스트 이미지 수: 843
🔎 [SUBMIT] TTA 예측 실행 (augment=True, conf 낮춰 최대한 수집)
[SUBMIT] 총 수집 박스: 7313
[SUBMIT] score >= 0.25 필터 후: 4116
💾 Saved CSV: E:\pytorch_env\ai05-level1-project\submission\submission_tta_conf_0.25.csv
📏 [LOCAL EVAL] Approx. mAP@[0.75:0.95] (NO TTA): 0.9800
📏 [LOCAL EVAL] Approx. mAP@[0.75:0.95] (with TTA): 0.5288
